### Importando módulbos e arquivos

In [1]:
import pandas as pd
import numpy as np
import requests
import time
import json
import re
from difflib import SequenceMatcher
from selenium import webdriver
from datetime import date
from bs4 import BeautifulSoup
from lxml import etree



In [2]:
lista_veiculos = []

def buscarDadosOlx(pages = 2, regiao = 'ES'):
    for x in range (0, pages):
        #url = 'https://es.olx.com.br/sul-do-espirito-santo/autos-e-pecas/carros-vans-e-utilitarios'
        url = 'https://es.olx.com.br/autos-e-pecas/carros-vans-e-utilitarios/fiat'
        
        if x == 0:
            #url = 'https://es.olx.com.br/sul-do-espirito-santo/autos-e-pecas/carros-vans-e-utilitarios'
            url = 'https://es.olx.com.br/autos-e-pecas/carros-vans-e-utilitarios/fiat'
        else:
            #url = 'https://es.olx.com.br/sul-do-espirito-santo/autos-e-pecas/carros-vans-e-utilitarios?o=' + str(x)
            url = 'https://es.olx.com.br/autos-e-pecas/carros-vans-e-utilitarios/fiat?o=' + str(x)
        # referer -> igual a url principal
        PARAMS = {
                    "authority": "es.olx.com.br",
                    "method": "GET",
                    "path": "/sul-do-espirito-santo/autos-e-pecas/carros-vans-e-utilitarios",
                    "scheme": "https",
                    "referer": "https://es.olx.com.br/autos-e-pecas/carros-vans-e-utilitarios/fiat",
                    "sec-fetch-mode": "navigate",
                    "sec-fech-site": "same-origin",
                    "sec-fetch-user": "?1",
                    "user-agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebkit/537.36 (KHTML, like Gecko) Chrome/79.0.394'
                 }
        page = requests.get(url = url, headers = PARAMS)
        soup = BeautifulSoup(page.content, 'lxml')
        anuncios = soup.find_all("li", {"class": "sc-1fcmfeb-2 fvbmlV"})
        
        for a in anuncios:
            try:
                urlVeiculo = a.find("a")["href"]
                
                preco_veiculo = a.find_all("span", class_="m7nrfa-0 cjhQnm sc-bdVaJa cpfGxa")[0].contents[0]
                preco_veiculo = preco_veiculo.split("R$")[1]
                preco_veiculo = float(preco_veiculo.replace(".",""))
                
                municipio_veiculo = a.find_all("span", class_ = "sc-1c3ysll-1 flPYFW sc-bdVaJa bxVNCd")[0].contents[0]
                try:
                    municipio_veiculo = municipio_veiculo.split(",")[0]
                except:
                    municipio_veiculo = municipio_veiculo
                
                PARAMS_V = {
                    "authority": "es.olx.com.br",
                    "method": "GET",
                    "path": "/sul-do-espirito-santo/autos-e-pecas/carros-vans-e-utilitarios",
                    "scheme": "https",
                    "referer": f'{urlVeiculo}',
                    "sec-fetch-mode": "navigate",
                    "sec-fech-site": "same-origin",
                    "sec-fetch-user": "?1",
                    "user-agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebkit/537.36 (KHTML, like Gecko) Chrome/79.0.394'
                 }
                
                page_v = requests.get(url = urlVeiculo, headers = PARAMS_V)
                soup_v = BeautifulSoup(page_v.content, 'lxml')
                #dom = etree.HTML(str(soup_v))
                detalhes = soup_v.find_all("div", {"class": "sc-hmzhuo HlNae sc-jTzLTM iwtnNi"})
                
                modelo_veiculo = ''
                dict_v = {}
                
                caracteristicas = {'Modelo': '', 
                                           'Marca': '', 
                                           'Tipo de veículo': '', 
                                           'Ano': '', 
                                           'Quilometragem': '', 
                                           'Potência do motor': '', 
                                           'Combustível': '', 
                                           'Kit GNV': '', 
                                           'Câmbio': '', 
                                           'Direção': '', 
                                           'Cor': '', 
                                           'Portas': ''}
                
                
                caracteristicas['Município'] = municipio_veiculo
                caracteristicas['Preço'] = preco_veiculo
                
                for b in detalhes:
                    try:
                        chave = b.find_all("span", class_ = "sc-ifAKCX dCObfG")[0].contents[0]
                        a_elemento = ['Modelo', 'Marca', 'Ano', 'Combustível']
                        json = {}
                        
                        for item in caracteristicas:
                            
                            if chave in a_elemento:
                                valor = b.find_all("a", class_ = "sc-57pm5w-0 XtcoW")[0].contents[0]
                                caracteristicas[f'{chave}'] = valor
                                #print(caracteristicas[f'{chave}'])
                            else:
                                valor = b.find_all("span", class_ = "sc-ifAKCX cmFKIN")[0].contents[0]
                                caracteristicas[f'{chave}'] = valor
                                #json.update({chave:valor})
                    except:
                        #print("2. Erro")
                        pass
                
                lista_veiculos.append(caracteristicas)
                    #print(lista_veiculos)
            except:
                #print("1. Erro")
                pass

In [3]:
buscarDadosOlx(pages=50)

#print(lista_veiculos)

veiculos_df = pd.DataFrame(lista_veiculos)

veiculos_df.to_excel("Veiculos.xlsx")